# 14-01 Docker 基础

容器化部署是现代化应用的标准。

## 1. Dockerfile 编写

In [ ]:
# Dockerfile
FROM node:20-alpine

WORKDIR /app

# 先复制依赖文件（利用缓存层）
COPY package*.json ./
RUN npm ci --only=production

# 再复制源代码
COPY . .

EXPOSE 3000

CMD ["node", "server.js"]

## 2. 常用命令

In [ ]:
# 构建镜像
# docker build -t myapp:1.0 .

# 运行容器
# docker run -d -p 3000:3000 --name myapp myapp:1.0

# 查看日志
# docker logs -f myapp

# 进入容器
# docker exec -it myapp sh

# 停止并删除
# docker stop myapp && docker rm myapp

## 3. Docker Compose

In [ ]:
# docker-compose.yml
version: '3.8'
services:
  app:
    build: .
    ports:
      - "3000:3000"
    environment:
      - NODE_ENV=production
      - DATABASE_URL=postgres://user:pass@db:5432/mydb
    depends_on:
      - db
      - redis
  
  db:
    image: postgres:15-alpine
    environment:
      - POSTGRES_USER=user
      - POSTGRES_PASSWORD=pass
      - POSTGRES_DB=mydb
    volumes:
      - postgres_data:/var/lib/postgresql/data
  
  redis:
    image: redis:7-alpine
    volumes:
      - redis_data:/data

volumes:
  postgres_data:
  redis_data:

## 4. 多阶段构建（优化镜像大小）

In [ ]:
# 构建阶段
FROM node:20-alpine AS builder
WORKDIR /app
COPY package*.json ./
RUN npm ci
COPY . .
RUN npm run build

# 运行阶段（更小）
FROM node:20-alpine
WORKDIR /app
COPY package*.json ./
RUN npm ci --only=production
COPY --from=builder /app/dist ./dist
EXPOSE 3000
CMD ["node", "dist/server.js"]

## OpenClaw 的部署

In [ ]:
# 查看 OpenClaw 的 Dockerfile
# cat Dockerfile

# 启动开发环境
# docker-compose up -d

# 查看容器状态
# docker-compose ps

## 练习

1. 为你的应用编写 Dockerfile
2. 用 Docker Compose 部署完整环境
3. 优化镜像大小到 100MB 以下